In [1]:
!pip install vibelogger

In [2]:
# 必要なライブラリをインポート
import asyncio
import json
import random
from pathlib import Path
from rich import print

# -------------------------------------------------------------------
# VibeCoding Logger (vibelogger) のモック
# 元のコードに含まれていたものをそのまま使用します。
# もし `pip install vibelogger` でインストールできる場合は、この部分は不要です。
# -------------------------------------------------------------------
try:
    # 実際のパッケージがあればそれを使用
    from vibelogger import create_file_logger
    print("Found and using actual 'vibelogger' package.")
except ImportError:
    # パッケージがない場合は、以下のモック（模擬）クラスを使用
    print("Could not find 'vibelogger' package. Using a mock logger.")
    class MockLogger:
        def __init__(self, name, log_dir):
            self.name = name
            self.log_file = Path(log_dir) / f"{name}.log"
            print(f"MockLogger initialized. Logs will be written to: {self.log_file}")

        def _log(self, level, operation=None, message=None, context=None, **kwargs):
            import datetime
            log_entry = {
                "timestamp": datetime.datetime.now(datetime.timezone.utc).isoformat(),
                "level": level,
                "operation": operation,
                "message": message,
                "context": context or {},
                **kwargs
            }
            # ログファイルに追記
            with open(self.log_file, "a", encoding="utf-8") as f:
                f.write(json.dumps(log_entry) + "\n")

            # コンソールにも出力
            print(json.dumps(log_entry, indent=2))
            return log_entry

        def info(self, operation=None, message=None, context=None, human_note=None, **kwargs):
            return self._log("INFO", operation=operation, message=message, context=context, human_note=human_note, **kwargs)

        def warning(self, operation=None, message=None, context=None, human_note=None, **kwargs):
            return self._log("WARNING", operation=operation, message=message, context=context, human_note=human_note, **kwargs)

        def error(self, operation=None, message=None, context=None, human_note=None, **kwargs):
            return self._log("ERROR", operation=operation, message=message, context=context, human_note=human_note, **kwargs)

        def debug(self, operation=None, message=None, context=None, human_note=None, **kwargs):
            return self._log("DEBUG", operation=operation, message=message, context=context, human_note=human_note, **kwargs)

        def success(self, operation=None, message=None, context=None, human_note=None, **kwargs):
            return self._log("SUCCESS", operation=operation, message=message, context=context, human_note=human_note, **kwargs)

    def create_file_logger(name):
      # Colabの作業ディレクトリを基準とします
        BASE_DIR = Path.cwd()
        OUTPUTS_DIR = BASE_DIR / "outputs"
        LOG_DIR = OUTPUTS_DIR / f"{name}_logs"
        LOG_DIR.mkdir(parents=True, exist_ok=True) # 親ディレクトリもまとめて作成
        print(f"Base directory set to: {BASE_DIR}")
        print(f"Log directory set to: {LOG_DIR}")
        return MockLogger(name, LOG_DIR)


# -------------------------------------------------------------------
# Loggerのセットアップ
# -------------------------------------------------------------------
vibe_logger = create_file_logger("colab_test")

# -------------------------------------------------------------------
# ログ生成ロジック (テスト用に変更)
# -------------------------------------------------------------------
async def generate_logs_for_test(num_logs: int = 10):
    """指定された回数だけログを生成し、コンソールに出力します。"""
    print(f"\n--- Generating {num_logs} log entries ---")

    operations = ["user_login", "db_query", "file_upload", "api_call", "data_processing"]
    users = ["Alice", "Bob", "Charlie", "David"]

    for i in range(num_logs):
        # ログ生成の間隔をランダムに設定
        await asyncio.sleep(random.uniform(0.2, 1.0))

        log_level = random.choices(["INFO", "WARNING", "ERROR", "SUCCESS", "DEBUG"], weights=[5, 2, 1, 3, 2], k=1)[0]
        operation = random.choice(operations)
        user = random.choice(users)

        print(f"\n[Log {i+1}/{num_logs}]")
        print(f"log_level = {log_level}")

        if log_level == "INFO":
            vibe_logger.info(
                operation=operation,
                message=f"{operation} started",
                context={"user": user, "status": "started"},
                human_note=f"AI: Monitor {operation} completion for user {user}"
            )
        elif log_level == "WARNING":
            vibe_logger.warning(
                operation=operation,
                message=f"High latency detected in {operation}",
                context={"user": user, "latency_ms": random.randint(500, 1000)},
                human_note=f"AI: Investigate performance bottlenecks in {operation} - consider caching or optimization"
            )
        elif log_level == "ERROR":
            vibe_logger.error(
                operation=operation,
                message=f"Failed to complete {operation}",
                context={"user": user, "error_code": random.randint(500, 504)},
                human_note=f"AI: Analyze {operation} failure patterns and suggest error handling improvements"
            )
        elif log_level == "SUCCESS":
            vibe_logger.info(
                operation=operation,
                message=f"{operation} completed successfully",
                context={"user": user, "duration_ms": random.randint(100, 400)},
                human_note=f"AI: Track {operation} performance metrics and identify optimization opportunities"
            )
        elif log_level == "DEBUG":
            vibe_logger.debug(
                operation=operation,
                message=f"Debugging {operation}",
                context={"user": user, "details": {"step": random.randint(1, 5)}},
                human_note=f"AI: Trace execution flow for {operation} to identify potential issues"
            )

# -------------------------------------------------------------------
# メイン実行ブロック
# -------------------------------------------------------------------
async def main():
    """テストを実行するためのメイン非同期関数"""
    # 生成したいログの数を指定してテスト関数を呼び出す
    await generate_logs_for_test(num_logs=5)
    print("\n--- Log generation test finished. ---")
    print(f"All generated logs are saved")

# Colab/Jupyter環境で非同期コードを実行
# nest_asyncioは、既に実行中のイベントループで再度asyncio.run()を呼び出すエラーを防ぎます。
import nest_asyncio
nest_asyncio.apply()

# テストを実行
asyncio.run(main())

Found and using actual 'vibelogger' package.

--- Generating 5 log entries ---

[Log 1/5]

log_level = WARNING

[Log 2/5]

...

log_level = SUCCESS

[Log 5/5]

log_level = INFO

--- Log generation test finished. ---

All generated logs are saved